In [1]:

#Importing Necessary Packages
from keras.models import Sequential
from keras.layers import Activation, MaxPooling2D, Dropout, LSTM, Flatten, Concatenate, TimeDistributed
import numpy as np

from keras.layers import Concatenate

from keras.layers.convolutional import Conv2D
import numpy as np
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import ipywidgets as widgets
from IPython.html.widgets import *
import pickle
import h5py 
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation, Dropout, Flatten, Dense, Lambda
from keras.layers import ELU
from keras.optimizers import Adam
import keras.backend.tensorflow_backend as KTF
inputShape=(100,100,2)


%matplotlib inline

Using TensorFlow backend.
C:\Users\hp\Anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
        #Creating a model with 4 Convolution layers.
        model = Sequential()
        model.add(Convolution2D(32, 8,8 ,border_mode='same', subsample=(4,4),input_shape=inputShape))
        model.add(Activation('relu'))
        model.add(Convolution2D(64, 8,8 ,border_mode='same',subsample=(4,4)))
        model.add(Activation('relu'))
        model.add(Convolution2D(128, 4,4,border_mode='same',subsample=(2,2)))
        model.add(Activation('relu'))
        model.add(Convolution2D(128, 2,2,border_mode='same',subsample=(1,1)))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(128))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(128))
        model.add(Dropout(0.5))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(100, 100,..., strides=(4, 4), padding="same")`
  


C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (8, 8), strides=(4, 4), padding="same")`
  after removing the cwd from sys.path.
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (4, 4), strides=(2, 2), padding="same")`
  
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (2, 2), strides=(1, 1), padding="same")`
  



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [5]:
    def preproc(frame1,frame2):
        frame1 = frame1[200:400]
        frame1 = cv2.resize(frame1, (0,0), fx = 0.4, fy=0.5)
        frame2 = frame2[200:400]
        frame2 = cv2.resize(frame2, (0,0), fx = 0.4, fy=0.5)
        flow = np.zeros_like(frame1)
        prev = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        nxt = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
        flow_data = cv2.calcOpticalFlowFarneback(prev, nxt,None, 0.4, 1, 12, 2, 8, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow_data[...,0], flow_data[...,1])
        flow[...,1] = 255
        flow[...,0] = ang*180/np.pi/2
        flow[...,2] = (mag *15).astype(int)
        return flow
    def prepare(video_file="C:\\Users\\hp\\Desktop\\Comma AI\\data\\train.mp4",shuffle = True):
        speed_data = np.loadtxt("C:\\Users\\hp\\Desktop\\Comma AI\\data\\train.txt", delimiter=",")
        print ("preprocessing data...")
        optflow_dir="C:\\Users\\hp\\Desktop\\Raktest"
        os.mkdir(optflow_dir)
            #Decode video frames
        vid = cv2.VideoCapture(video_file)
        frame_cnt = vid.get(cv2.CAP_PROP_FRAME_COUNT)
        processed_video = np.empty((int(frame_cnt-1),int(100),int(100),3))
        ret, prev = vid.read()
        i = 0
        while True:
            ret, nxt  = vid.read()
            if not ret: 
                break
            flow = optflow(prev,nxt)
            prev = nxt
            flow = cv2.resize(flow, (100,100), interpolation = cv2.INTER_AREA)
            processed_video[i] = flow/127.5 - 1.0
            cv2.imwrite(optflow_dir + '/' + str(i) + ".png", flow)
            sys.stdout.write("\rProcessed " + str(i) + " frames" )
            i+=1
        if(shuffle):
            randomize = np.arange(len(processed_video))
            np.random.shuffle(randomize)
            processed_video = processed_video[randomize]
            speed_data = speed_data[randomize]

        return (processed_video, speed_data)

# The above code is used to preprocess the training data using optical flow. I have already done the preprocessing and saved it as a numpy file during experimentation

In [6]:
#loading the training data and it's labels
from numpy import asarray
from numpy import save
from numpy import load

X = load('dataX.npy')
Y = load('dataY.npy')

In [7]:
#training the model for 50 epochs
model.fit(X, Y, batch_size=64,
                    nb_epoch=50,validation_split=0.30)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  



Train on 14279 samples, validate on 6120 samples
Epoch 1/50
14279/14279 [==============================] - ETA: 4:14 - loss: 194.259 - ETA: 2:27 - loss: 201.624 - ETA: 1:51 - loss: 186.384 - ETA: 1:37 - loss: 168.029 - ETA: 1:27 - loss: 159.180 - ETA: 1:19 - loss: 147.056 - ETA: 1:13 - loss: 140.686 - ETA: 1:08 - loss: 136.386 - ETA: 1:04 - loss: 133.510 - ETA: 1:01 - loss: 127.713 - ETA: 59s - loss: 123.738 - ETA: 57s - loss: 121.34 - ETA: 55s - loss: 120.38 - ETA: 53s - loss: 117.90 - ETA: 52s - loss: 115.90 - ETA: 51s - loss: 115.41 - ETA: 50s - loss: 116.99 - ETA: 49s - loss: 115.53 - ETA: 48s - loss: 113.19 - ETA: 47s - loss: 111.90 - ETA: 46s - loss: 110.97 - ETA: 45s - loss: 109.12 - ETA: 45s - loss: 107.96 - ETA: 44s - loss: 106.88 - ETA: 43s - loss: 106.26 - ETA: 43s - loss: 105.57 - ETA: 42s - loss: 104.22 - ETA: 42s - loss: 103.19 - ETA: 41s - loss: 101.78 - ETA: 41s - loss: 100.33 - ETA: 40s - loss: 99.5060 - ETA: 40s - loss: 98.605 - ETA: 40s - loss: 97.085 - ETA: 39s - l

14279/14279 [==============================] - ETA: 36s - loss: 28.300 - ETA: 38s - loss: 35.262 - ETA: 37s - loss: 39.395 - ETA: 37s - loss: 35.621 - ETA: 38s - loss: 33.627 - ETA: 38s - loss: 33.045 - ETA: 38s - loss: 33.134 - ETA: 38s - loss: 30.516 - ETA: 38s - loss: 30.355 - ETA: 38s - loss: 29.129 - ETA: 38s - loss: 28.383 - ETA: 38s - loss: 27.734 - ETA: 37s - loss: 26.788 - ETA: 37s - loss: 26.319 - ETA: 37s - loss: 25.615 - ETA: 37s - loss: 25.052 - ETA: 37s - loss: 24.463 - ETA: 36s - loss: 23.661 - ETA: 36s - loss: 23.529 - ETA: 36s - loss: 23.637 - ETA: 36s - loss: 23.160 - ETA: 36s - loss: 22.790 - ETA: 35s - loss: 22.633 - ETA: 35s - loss: 22.182 - ETA: 35s - loss: 21.954 - ETA: 35s - loss: 21.893 - ETA: 35s - loss: 21.658 - ETA: 35s - loss: 21.421 - ETA: 34s - loss: 21.370 - ETA: 34s - loss: 20.988 - ETA: 34s - loss: 20.583 - ETA: 34s - loss: 20.254 - ETA: 34s - loss: 20.191 - ETA: 33s - loss: 20.267 - ETA: 33s - loss: 20.082 - ETA: 33s - loss: 19.857 - ETA: 33s - loss: 

14279/14279 [==============================] - ETA: 38s - loss: 9.49 - ETA: 39s - loss: 11.068 - ETA: 38s - loss: 10.992 - ETA: 38s - loss: 10.902 - ETA: 38s - loss: 11.257 - ETA: 37s - loss: 11.029 - ETA: 37s - loss: 10.735 - ETA: 37s - loss: 10.190 - ETA: 37s - loss: 10.043 - ETA: 37s - loss: 10.517 - ETA: 37s - loss: 10.299 - ETA: 37s - loss: 10.036 - ETA: 37s - loss: 10.506 - ETA: 37s - loss: 10.783 - ETA: 37s - loss: 10.723 - ETA: 37s - loss: 10.487 - ETA: 38s - loss: 10.530 - ETA: 39s - loss: 10.306 - ETA: 39s - loss: 10.148 - ETA: 39s - loss: 9.949 - ETA: 39s - loss: 10.063 - ETA: 39s - loss: 10.083 - ETA: 39s - loss: 10.150 - ETA: 39s - loss: 10.081 - ETA: 39s - loss: 10.135 - ETA: 39s - loss: 10.181 - ETA: 39s - loss: 10.233 - ETA: 38s - loss: 10.268 - ETA: 38s - loss: 10.462 - ETA: 38s - loss: 10.444 - ETA: 37s - loss: 10.388 - ETA: 37s - loss: 10.455 - ETA: 37s - loss: 10.508 - ETA: 37s - loss: 10.424 - ETA: 37s - loss: 10.403 - ETA: 37s - loss: 10.500 - ETA: 36s - loss: 10.

14279/14279 [==============================] - ETA: 38s - loss: 13.398 - ETA: 40s - loss: 11.461 - ETA: 40s - loss: 11.478 - ETA: 40s - loss: 11.768 - ETA: 39s - loss: 10.686 - ETA: 39s - loss: 10.323 - ETA: 39s - loss: 10.217 - ETA: 39s - loss: 10.347 - ETA: 39s - loss: 10.350 - ETA: 39s - loss: 10.191 - ETA: 38s - loss: 10.164 - ETA: 38s - loss: 10.012 - ETA: 38s - loss: 9.881 - ETA: 38s - loss: 10.119 - ETA: 38s - loss: 9.948 - ETA: 37s - loss: 9.78 - ETA: 37s - loss: 9.84 - ETA: 37s - loss: 9.65 - ETA: 37s - loss: 9.53 - ETA: 37s - loss: 9.54 - ETA: 37s - loss: 9.76 - ETA: 37s - loss: 9.66 - ETA: 37s - loss: 9.65 - ETA: 36s - loss: 9.46 - ETA: 36s - loss: 9.64 - ETA: 36s - loss: 9.51 - ETA: 36s - loss: 9.49 - ETA: 36s - loss: 9.45 - ETA: 35s - loss: 9.51 - ETA: 35s - loss: 9.45 - ETA: 35s - loss: 9.41 - ETA: 35s - loss: 9.44 - ETA: 35s - loss: 9.59 - ETA: 34s - loss: 9.53 - ETA: 34s - loss: 9.46 - ETA: 34s - loss: 9.41 - ETA: 34s - loss: 9.40 - ETA: 34s - loss: 9.42 - ETA: 33s - lo

...

14279/14279 [==============================] - ETA: 34s - loss: 2.74 - ETA: 36s - loss: 3.42 - ETA: 39s - loss: 3.74 - ETA: 38s - loss: 4.38 - ETA: 37s - loss: 4.74 - ETA: 37s - loss: 4.89 - ETA: 36s - loss: 4.96 - ETA: 36s - loss: 5.24 - ETA: 37s - loss: 5.53 - ETA: 36s - loss: 5.40 - ETA: 36s - loss: 5.28 - ETA: 36s - loss: 5.19 - ETA: 35s - loss: 5.17 - ETA: 35s - loss: 5.18 - ETA: 35s - loss: 5.17 - ETA: 35s - loss: 5.09 - ETA: 35s - loss: 5.07 - ETA: 34s - loss: 5.06 - ETA: 34s - loss: 5.08 - ETA: 34s - loss: 5.03 - ETA: 34s - loss: 4.96 - ETA: 34s - loss: 4.91 - ETA: 34s - loss: 4.92 - ETA: 33s - loss: 4.92 - ETA: 33s - loss: 4.96 - ETA: 33s - loss: 4.87 - ETA: 33s - loss: 4.81 - ETA: 33s - loss: 4.85 - ETA: 32s - loss: 4.81 - ETA: 32s - loss: 4.73 - ETA: 32s - loss: 4.75 - ETA: 32s - loss: 4.75 - ETA: 32s - loss: 4.75 - ETA: 31s - loss: 4.75 - ETA: 31s - loss: 4.76 - ETA: 31s - loss: 4.70 - ETA: 31s - loss: 4.67 - ETA: 31s - loss: 4.65 - ETA: 30s - loss: 4.65 - ETA: 30s - loss: 

14279/14279 [==============================] - ETA: 38s - loss: 3.43 - ETA: 37s - loss: 3.48 - ETA: 37s - loss: 3.80 - ETA: 37s - loss: 3.46 - ETA: 37s - loss: 3.51 - ETA: 37s - loss: 3.78 - ETA: 37s - loss: 3.77 - ETA: 37s - loss: 3.76 - ETA: 37s - loss: 3.92 - ETA: 37s - loss: 3.84 - ETA: 37s - loss: 3.95 - ETA: 36s - loss: 3.87 - ETA: 36s - loss: 3.87 - ETA: 36s - loss: 3.85 - ETA: 36s - loss: 3.81 - ETA: 36s - loss: 3.79 - ETA: 36s - loss: 3.79 - ETA: 36s - loss: 3.77 - ETA: 35s - loss: 3.72 - ETA: 35s - loss: 3.65 - ETA: 35s - loss: 3.75 - ETA: 35s - loss: 3.71 - ETA: 35s - loss: 3.68 - ETA: 35s - loss: 3.72 - ETA: 34s - loss: 3.64 - ETA: 34s - loss: 3.71 - ETA: 34s - loss: 3.73 - ETA: 34s - loss: 3.69 - ETA: 34s - loss: 3.68 - ETA: 34s - loss: 3.75 - ETA: 34s - loss: 3.79 - ETA: 33s - loss: 3.85 - ETA: 33s - loss: 3.82 - ETA: 33s - loss: 3.79 - ETA: 33s - loss: 3.83 - ETA: 33s - loss: 3.80 - ETA: 32s - loss: 3.79 - ETA: 32s - loss: 3.78 - ETA: 32s - loss: 3.81 - ETA: 32s - loss: 

14279/14279 [==============================] - ETA: 36s - loss: 3.93 - ETA: 36s - loss: 3.67 - ETA: 36s - loss: 4.29 - ETA: 36s - loss: 4.61 - ETA: 36s - loss: 4.66 - ETA: 36s - loss: 4.69 - ETA: 36s - loss: 4.65 - ETA: 36s - loss: 4.76 - ETA: 36s - loss: 4.84 - ETA: 36s - loss: 4.68 - ETA: 35s - loss: 4.56 - ETA: 35s - loss: 4.65 - ETA: 35s - loss: 4.81 - ETA: 35s - loss: 4.65 - ETA: 35s - loss: 4.56 - ETA: 35s - loss: 4.57 - ETA: 35s - loss: 4.48 - ETA: 35s - loss: 4.45 - ETA: 34s - loss: 4.40 - ETA: 34s - loss: 4.36 - ETA: 34s - loss: 4.32 - ETA: 34s - loss: 4.27 - ETA: 34s - loss: 4.22 - ETA: 34s - loss: 4.25 - ETA: 33s - loss: 4.23 - ETA: 33s - loss: 4.26 - ETA: 33s - loss: 4.20 - ETA: 33s - loss: 4.21 - ETA: 33s - loss: 4.25 - ETA: 33s - loss: 4.22 - ETA: 32s - loss: 4.24 - ETA: 32s - loss: 4.25 - ETA: 32s - loss: 4.21 - ETA: 32s - loss: 4.22 - ETA: 32s - loss: 4.27 - ETA: 32s - loss: 4.32 - ETA: 32s - loss: 4.30 - ETA: 31s - loss: 4.35 - ETA: 31s - loss: 4.34 - ETA: 31s - loss: 

14279/14279 [==============================] - ETA: 37s - loss: 3.69 - ETA: 36s - loss: 3.68 - ETA: 38s - loss: 3.83 - ETA: 37s - loss: 4.23 - ETA: 37s - loss: 4.15 - ETA: 37s - loss: 4.06 - ETA: 37s - loss: 4.06 - ETA: 37s - loss: 4.04 - ETA: 37s - loss: 4.07 - ETA: 37s - loss: 4.07 - ETA: 37s - loss: 4.06 - ETA: 36s - loss: 3.96 - ETA: 36s - loss: 4.02 - ETA: 36s - loss: 4.02 - ETA: 36s - loss: 4.06 - ETA: 36s - loss: 4.09 - ETA: 35s - loss: 4.22 - ETA: 35s - loss: 4.29 - ETA: 35s - loss: 4.40 - ETA: 35s - loss: 4.46 - ETA: 35s - loss: 4.44 - ETA: 35s - loss: 4.43 - ETA: 34s - loss: 4.42 - ETA: 34s - loss: 4.44 - ETA: 34s - loss: 4.45 - ETA: 34s - loss: 4.47 - ETA: 34s - loss: 4.46 - ETA: 33s - loss: 4.52 - ETA: 33s - loss: 4.54 - ETA: 33s - loss: 4.51 - ETA: 33s - loss: 4.50 - ETA: 33s - loss: 4.49 - ETA: 33s - loss: 4.46 - ETA: 32s - loss: 4.44 - ETA: 32s - loss: 4.42 - ETA: 32s - loss: 4.40 - ETA: 32s - loss: 4.37 - ETA: 32s - loss: 4.36 - ETA: 32s - loss: 4.36 - ETA: 31s - loss: 

14279/14279 [==============================] - ETA: 36s - loss: 5.20 - ETA: 36s - loss: 4.51 - ETA: 36s - loss: 4.39 - ETA: 36s - loss: 4.43 - ETA: 36s - loss: 4.26 - ETA: 37s - loss: 4.16 - ETA: 36s - loss: 4.35 - ETA: 36s - loss: 4.39 - ETA: 36s - loss: 4.21 - ETA: 36s - loss: 4.18 - ETA: 36s - loss: 4.15 - ETA: 36s - loss: 4.07 - ETA: 36s - loss: 3.93 - ETA: 35s - loss: 3.99 - ETA: 35s - loss: 3.97 - ETA: 35s - loss: 3.95 - ETA: 35s - loss: 3.91 - ETA: 35s - loss: 3.97 - ETA: 35s - loss: 3.99 - ETA: 34s - loss: 3.99 - ETA: 34s - loss: 3.96 - ETA: 34s - loss: 4.06 - ETA: 34s - loss: 3.99 - ETA: 34s - loss: 3.99 - ETA: 34s - loss: 3.97 - ETA: 33s - loss: 3.99 - ETA: 33s - loss: 3.97 - ETA: 33s - loss: 4.02 - ETA: 33s - loss: 4.13 - ETA: 33s - loss: 4.10 - ETA: 33s - loss: 4.10 - ETA: 32s - loss: 4.06 - ETA: 32s - loss: 4.05 - ETA: 32s - loss: 4.07 - ETA: 32s - loss: 4.02 - ETA: 32s - loss: 3.98 - ETA: 32s - loss: 3.96 - ETA: 31s - loss: 4.01 - ETA: 31s - loss: 4.00 - ETA: 31s - loss: 

In [9]:
#saving the model fo future use
model.save('comma.h5')

In [31]:
import sys
from sklearn.metrics import mean_squared_error
err2=[]
print ("Starting testing")
#loading the preprocessed test video from a numpy file
A = load('dataA.npy')

if True:
    #predicting the values for test.txt
    for x in A:
        
        pred_y = model.predict(np.array([x[:,:,[0,2]]]))[0,0]
        err2.append(pred_y)
        
    print ("Done predicting")

Starting testing
Done predicting


In [35]:
len(err2)

10797

In [34]:
#writing the predictions in test.txt
with open('test.txt', 'w') as f:
    for item in err2:
        f.write("%s\n" % item)

In [33]:
        A = load('dataX.npy')
        
        B = load('dataY.npy')
        print ("Checking the MSE")
        print (model.evaluate(A,B))
        #The MSE value on Training data is 1.117


Checking the MSE
20399/20399 [==============================] - ETA: 21 - ETA: 22 - ETA: 23 - ETA: 23 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 1